In [51]:
import pandas as pd
import numpy as np
import os
from shapely.geometry import Point, Polygon

cwd =os.getcwd()

### Structure socioeconomic data by year

In [42]:
socioeco_df

,POSTNR,POSTNAVN,KOMKODE,KOMNAVN,YEAR,SALGSPRIS,DISINDK,LANGLEDIG,KRIMINELITET,LAVINDK,SKILSMISSER,ANDEL_INDV
0,1000,København K,101,Københavns Kommune,2011,23794.750000,189182,1.966667,4.8,36.7,21.8,14.729180
1,1050,København K,101,Københavns Kommune,2011,23794.750000,189182,1.966667,4.8,36.7,21.8,14.729180
2,1051,København K,101,Københavns Kommune,2011,23794.750000,189182,1.966667,4.8,36.7,21.8,14.729180
3,1052,København K,101,Københavns Kommune,2011,23794.750000,189182,1.966667,4.8,36.7,21.8,14.729180
4,1053,København K,101,Københavns Kommune,2011,23794.750000,189182,1.966667,4.8,36.7,21.8,14.729180
...,...,...,...,...,...,...,...,...,...,...,...,...
4259,9970,Strandby,813,Frederikshavn Kommune,2019,6252.666667,214560,0.650000,4.2,19.2,12.9,4.298998
4260,9981,Jerup,813,Frederikshavn Kommune,2019,8695.750000,214560,0.650000,4.2,19.2,12.9,4.298998
4261,9982,Ålbæk,813,Frederikshavn Kommune,2019,6225.000000,214560,0.650000,4.2,19.2,12.9,4.298998
4262,9982,Ålbæk,860,Hjørring Kommune,2019,6225.000000,217772,0.483333,4.2,20.5,11.4,5.103595


In [43]:
socioeco_df = pd.read_excel(cwd+'/socioeconomic_data/socioeco_data.xlsx', sheet_name = 'Samlet')

postnr_columns =['POSTNR', 'YEAR', 'KOMKODE', 'SALGSPRIS']
kom_columns = ['KOMKODE', 'YEAR', 'DISINDK', 'KRIMINELITET', 'LAVINDK', 'SKILSMISSER', 'ANDEL_INDV']

post_df = socioeco_df[postnr_columns]
kom_df = socioeco_df[kom_columns]

years = [2011, 2015, 2019]
post_split = [post_df[post_df['YEAR'] == year] for year in years]
kom_split = [kom_df[kom_df['YEAR'] == year] for year in years]

In [44]:
post_split[2]

,POSTNR,YEAR,KOMKODE,SALGSPRIS
2841,1000,2019,101,37082.250000
2842,1050,2019,101,37082.250000
2843,1051,2019,101,37082.250000
2844,1052,2019,101,37082.250000
2845,1053,2019,101,37082.250000
...,...,...,...,...
4259,9970,2019,813,6252.666667
4260,9981,2019,813,8695.750000
4261,9982,2019,813,6225.000000
4262,9982,2019,860,6225.000000


### Merge school data and socioeconomic data by postal code and kommune

In [45]:
df_list = []
for data, year in zip(post_split, [2011, 2015, 2019]):
    # load data
    soc_data = data
    school_data = pd.read_csv(cwd+f'/school_data/school_clean_{year-1}-{year}.csv', index_col=0)

    # merge data
    merged = pd.merge(left = school_data, right = soc_data, left_on = ['beliggenhedskommunenr', 'postnr'], right_on = ['KOMKODE', 'POSTNR'])

    # format
    merged.columns = merged.columns.str.lower()
    merged = merged.T.drop_duplicates().T

    merged.to_csv(f'postnr_background_vars{year}.csv')

    df_list.append(merged)

for data, year in zip(kom_split, [2011, 2015, 2019]):
    data.drop_duplicates()\
        .to_csv(f'kom_background_vars{year}.csv')

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
import pyproj


In [2]:
# Well, that was horrible...
# conda remove pyproj
# pip install pyproj
# conda install geopandas 

# Læs baggrundsvariable of oversæt til utm
df = pd.read_csv('postnr_background_vars2019.csv', index_col=0)

c = [Point(x, y) for x, y in zip(df['lon'], df['lat'])]
gdf = gpd.GeoDataFrame(df, geometry = c)

gdf.crs = {'init': 'epsg:4326'}
gdf_utm = gdf.to_crs(epsg = 32632)

# frasorter efterskoler (behold privat- og friskoler for at undgå for stort fald i antal observationer)
gdf_utm = gdf_utm[gdf_utm['institutionstype2'] == 'Grundskoler']

In [5]:
# Læs valgdistrikter og tilføj centroid
valg_distrikt = gpd.read_file(cwd+'\scrape_geodata\geodata\dagi_10m_nohist_l1.afstemningsomraade\\afstemningsomraade.shp', driver = 'ESRI Shapefile')
valg_distrikt['centroid']=valg_distrikt['geometry'].centroid

In [6]:
valg_geo = valg_distrikt.set_geometry('centroid')
schools_geo = gdf_utm.geometry.unary_union

In [7]:
# Find for hver centroid den nærmeste skole
# For valg_distrikt['c½entroid'] find nærmeste gdf_utm['geometry'] ogtilføj tilhørende 'institutionsnummer'
pts3 = gdf_utm.geometry.unary_union
def near(point, pts = pts3):
    nearest = gdf_utm.geometry == nearest_points(point, pts)[1]
    return gdf_utm[nearest]['institutionsnummer'].values[0]
    
valg_geo['institutionsnummer'] = valg_geo.apply(lambda row: near(row.geometry), axis = 1)

In [51]:
"""test_valg = valg_geo.iloc[0].centroid

nearest = nearest_points(test_valg, schools_geo)[1]
print(test_valg)
print(nearest)
gdf_utm[gdf_utm['geometry'] == nearest]"""

"test_valg = valg_geo.iloc[0].centroid\n\nnearest = nearest_points(test_valg, schools_geo)[1]\nprint(test_valg)\nprint(nearest)\ngdf_utm[gdf_utm['geometry'] == nearest]"

In [8]:
# merge election to school in 'intitutionsnummer
merge1 = pd.merge(left=valg_geo, right = gdf_utm, on = 'institutionsnummer')
merge1.columns

Index(['objectid', 'id_lokalid', 'id_namespa', 'status', 'geometrist',
       'virkningfr', 'virkningti', 'virkningsa', 'forretning', 'registreri',
       'registre00', 'registre01', 'forretni00', 'forretni01', 'dagiid',
       'navn_x', 'redigering', 'dataspecif', 'landekode', 'skala',
       'afstemning', 'afstemni00', 'afstemni01', 'kommunekod', 'kommunelok',
       'opstilling', 'opstilli00', 'udtraeksda', 'gmlid', 'geometry_x',
       'centroid', 'institutionsnummer', 'karakter', 'soc_ref',
       'beliggenhedskommunenr', 'lat', 'lon', 'postnr', 'navn_y',
       'institutionstype2', 'institutionstype3', 'coordinates', 'year',
       'salgspris', 'geometry_y'],
      dtype='object')

In [9]:
# merge merge1 to mapping on 'objectid'
mapping = pd.read_csv(cwd+'/scrape_geodata/mapping_geodata_v3.csv', sep =';')
mapping.objectid = mapping.objectid.astype(str)
merge2 = pd.merge(left = merge1, right = mapping[['objectid', 'refs']], on = 'objectid')
merge2

,objectid,id_lokalid,id_namespa,status,geometrist,virkningfr,virkningti,virkningsa,forretning,registreri,...,lon,postnr,navn_y,institutionstype2,institutionstype3,coordinates,year,salgspris,geometry_y,refs
0,1285591,711178,http://data.gov.dk/dagi,vedtaget,endelig,20181127000000,None,Andet,systemetablering,20181127115950.448,...,8.300398,7620,Christinelystskolen,Grundskoler,Folkeskoler,"(56.5417072334074, 8.300397891210551)",2019,6057.50,POINT (456980.040 6266591.920),F901665003.htm
1,1285619,711172,http://data.gov.dk/dagi,vedtaget,endelig,20181127000000,None,Andet,systemetablering,20181127115950.448,...,8.300398,7620,Christinelystskolen,Grundskoler,Folkeskoler,"(56.5417072334074, 8.300397891210551)",2019,6057.50,POINT (456980.040 6266591.920),F901665004.htm
2,1285631,706536,http://data.gov.dk/dagi,vedtaget,endelig,20181127000000,None,Andet,systemetablering,20181127115950.448,...,8.300398,7620,Christinelystskolen,Grundskoler,Folkeskoler,"(56.5417072334074, 8.300397891210551)",2019,6057.50,POINT (456980.040 6266591.920),F901665001.htm
3,1295999,706714,http://data.gov.dk/dagi,vedtaget,endelig,20190429000000,None,Andet,systemetablering,20190429145414.863,...,9.723181,5500,Fænøsund Friskole,Grundskoler,Friskoler og private grundskoler,"(55.499902107147705, 9.72318121375711)",2019,13502.25,POINT (545683.475 6150659.929),F605410001.htm
4,1285601,701882,http://data.gov.dk/dagi,vedtaget,endelig,20181127000000,None,Andet,systemetablering,20181127115950.448,...,8.178329,7673,Harboøre Skole og Børnecenter,Grundskoler,Folkeskoler,"(56.6178540439447, 8.17832922896946)",2019,7482.00,POINT (449575.430 6275150.450),F901665008.htm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1379,1303228,712204,http://data.gov.dk/dagi,vedtaget,endelig,20190612000000,None,Andet,systemetablering,20190612113851.354,...,11.746338,4700,Herlufsholm Skole og Gods,Grundskoler,Friskoler og private grundskoler,"(55.2458162, 11.74633787)",2019,12126.50,POINT (674580.150 6125584.880),F504370026.htm
1380,1306341,703948,http://data.gov.dk/dagi,vedtaget,endelig,20191217000000,None,Andet,systemetablering,20191217141245.217,...,10.113277,8381,Tilst Skole,Grundskoler,Folkeskoler,"(56.189536015882204, 10.113277148416598)",2019,18300.25,POINT (569090.820 6227732.500),F802751028.htm
1381,1306371,709972,http://data.gov.dk/dagi,vedtaget,endelig,20191217000000,None,Andet,systemetablering,20191217141245.217,...,10.229043,8240,Risskov Skole,Grundskoler,Folkeskoler,"(56.192965125566595, 10.2290432602524)",2019,30179.00,POINT (576268.150 6228236.160),F804751045.htm
1382,1306648,702938,http://data.gov.dk/dagi,vedtaget,endelig,20200101000000,None,Geodatastyrelsen,ændretBekendtgørelse,20200108132505.108,...,9.346793,8800,Hald Ege Fri- og Efterskole,Grundskoler,Friskoler og private grundskoler,"(56.411417510544005, 9.34679331044587)",2019,12405.75,POINT (521398.360 6251924.723),F903791002.htm


In [10]:
# merge to election results
election_result = pd.read_csv(cwd+'/scrape_geodata/data2019.csv')

merge3 = pd.merge(left = merge2, right = election_result, on = 'refs')
merge3.kommunekod = merge3.kommunekod.astype(int).astype(str)
merge3.kommunekod

0        665
1        665
2        665
3        665
4        665
        ... 
19510    480
19511    480
19512    480
19513    480
19514    480
Name: kommunekod, Length: 19515, dtype: object

In [11]:
# merge to background vars on kommune level
background = pd.read_csv('kom_background_vars2019.csv', index_col = 0)

background.KOMKODE = background.KOMKODE.astype(str)
background

all_2019 = pd.merge(left = merge3, right = background, left_on = 'kommunekod', right_on = 'KOMKODE')
all_2019.to_csv('all_2019.csv')


In [102]:
all_2019.columns = all_2019.columns.str.lower()

col_vars = ['objectid','navn_x', 'afstemni00','kommunekod', 'institutionsnummer', 'karakter', 'soc_ref', 'postnr', 'navn_y', 'institutionstype2', 'institutionstype3', 'year_x', 'salgspris', 'party', 'votes', 'eligible_count', 'blank', 'invalid', 'total_valid', 'total_casted', 'time_stamp', 'names', 'constituency', 'voting_area', 'municipality', 'disindk', 'kriminelitet', 'lavindk', 'skilsmisser', 'andel_indv']

all_2019[all_2019['refs'] == 'F809746006.htm'][['party', 'votes']]

,party,votes
13222,Socialdemokratiet,1.659
13223,Radikale Venstre,547.000
13224,Det Konservative Folkeparti,365.000
13225,Nye Borgerlige,137.000
13226,Klaus Riskær Pedersen,51.000
13227,SF - Socialistisk Folkeparti,534.000
13228,Liberal Alliance,150.000
13229,Kristendemokraterne,103.000
13230,Dansk Folkeparti,457.000
13231,Stram Kurs,85.000


In [103]:
temp_2019 = all_2019[col_vars]

In [104]:
temp_2019[temp_2019['objectid'] == '1462182'][['party', 'votes']]
# 1275543

,party,votes
11895,Socialdemokratiet,1.555
11896,Radikale Venstre,1.099
11897,Det Konservative Folkeparti,494.000
11898,Nye Borgerlige,109.000
11899,Klaus Riskær Pedersen,85.000
11900,SF - Socialistisk Folkeparti,734.000
11901,Liberal Alliance,359.000
11902,Kristendemokraterne,90.000
11903,Dansk Folkeparti,270.000
11904,Stram Kurs,130.000


## Fix the absolute shitshow that is the vote count formatting

In [337]:
# We are facing an issue with the formatting of the scraped data. Because of the separator '.', when we load the data, python thinks that every number larger than 1000 is actually decimal. We need to identify, which numbers need to be multiplied by 1000. To do this, we find modulus 1 for each vote count, and if this is different from 0, we should multiply the number by 1000. There is however one issue with this approach: If a party has received exactly 1000 (or 2000 or 3000 etc) votes at a polling place, our data reports the number of votes to be 1.0. Before making the conversion, we need to check if this is the case. At some smaller polling places, however, some parties may actually have received only 1 vote. To get around this, we first check manually for the maximum voting count by party, given that modulus 1 of the voting count is different from 0, to find the maximum number of votes for each party. Then, if the maximum voting count is, say, 4792, we check the KMD website, for each of the polling places, that report a voting count of 1, 2, 3 and 4. If any of these voting counts are actually 1000 etc., they will be multiplied by 1000. We do this for every praty, that have received more than 1000 votes at a single polling place. Then we identify observations with modulus 1 different from zero and multiply these by 1000. 
# There is a risk of missing some observations, in case the maximum voting count for a party is a multiple of 1000. In that case, we won't recognise the observation as one that should be multiplied by 1000. This is, however, unlikely enough for us to assume it won't be the case.

# Ideally, when we scrape, we should store the data as strings, so we can delete the seperators before converting them to ints. We should try and fix this, the next time we scrape. If not, this method should be our fall back.


# max number of votes for A: 4.792  (4792) objectid 1306109
# max number of votes for V: 4.556 (4556) objectid 1306030
# max number of votes for B: 2.448 (2448) objectid 1306375
# max number of votes for C: 3.218 (3218) objectid 1306109
# max number of votes for D: no problem
# max number of votes for E: no problem
# max number of votes for F: 1.775 (1775) objectid 1306109
# max number of votes for I: no problem
# max number of votes for K: 1.114 (1114) objectid 1298885
# max number of votes for O: 1.104 (1104) objectid 1289687
# max number of votes for P: no problem
# max number of votes for Ø: 2.608 (2608) objectid 1304839
# måx number of votes for Å: 1.142 (1142)

# Check for problem parties (manually on kmd) if any af the parties have received exactly 1000, 2000 etc votes.
#  Limits:
# Venstre and Socialdemokratiet: 5
# Konservative: 4
# Radikale Venstre, Enhedslisten: 3
# SF, Kristendemokraterne, Dansk Folkeparti, Alternativet: 2

# 40 observations have to be checked manually.
# 3 observations are reporter as 1.0 or 2.0 but should be multiplied by 1000.

In [96]:
# Template for checking max number of votes:
temp_2019[temp_2019['votes']%1 != 0][temp_2019['party'] == 'Socialdemokratiet'][['party','objectid','votes']].sort_values(by = 'votes')

# Finding observations to check with kmd
limit5 = ['Venstre, Danmarks Liberale Parti', 'Socialdemokratiet']
limit4 = ['Det Konservative Folkeparti']
limit3 = ['Radikale Venstre', 'Enhedslisten']
limit2 = ['SF - Socialistisk Folkeparti', 'Kristendemokraterne', 'Dansk Folkeparti', 'Alternativet']

limits = {5: limit5, 4: limit4, 3: limit3, 2:limit2}

obs_to_check = []

for limit, parties in limits.items():
    for party in parties:
        df = temp_2019[temp_2019['votes']<limit][temp_2019['party'] == party][temp_2019['votes']%1==0]
        obs_to_check.append(df)

check_df = pd.concat(obs_to_check)

# To inspect and check with kmd
check_df.sort_values(by = 'constituency')[['municipality','constituency', 'navn_x','party','votes']]
obs_votes_1000 = [8198, 12558, 873] 

In [97]:
# For some reason i can't get pandas to update the votes inplace, so now i'm taking a detour..
temp_2019['vote_multiplyer'] = np.where(temp_2019['votes']%1 != 0, 1000, 1)

temp_2019.loc[obs_votes_1000,'vote_multiplyer'] = 1000
temp_2019.loc[obs_votes_1000]['vote_multiplyer']

8198     1000
12558    1000
873      1000
Name: vote_multiplyer, dtype: int32

In [101]:
temp_2019['votes_v2'] = temp_2019['votes']*temp_2019['vote_multiplyer']
temp_2019[temp_2019['party'] == 'Det Konservative Folkeparti']['votes_v2'].max()

3218.0

In [ ]:
votes_df.groupby(by = 'objectid').sum()
votes_df.sort_values(by = 'votes')

In [57]:
# Find centroids pr valgsted DONE
# Find for hver centroid den nærmeste skole DONE
# Lav column med institutionsnummer for nærmeste skole DONE
# Merge valgdistrikter med skoledata på institutionsnummer DONE
# Merge på mapping fil på objectid DONE
# merge på valgresultater fra 2019 på refs DONE
# split i kommune og postnnr DONE
# ryd op i data - kollaps valgresultater
# ryd op i kode
# tilføj huspriser for kbhk